In [67]:
import sys

sys.path.append('E:/UADY/CARLA/CARLA_Latest/WindowsNoEditor/PythonAPI/carla')

import carla
import numpy as np
import re
import cv2

In [68]:
client = carla.Client('localhost', 2000)
world = client.get_world()

RuntimeError: trying to operate on a destroyed actor; an actor's function was called, but the actor is already destroyed.

In [4]:
client.load_world('Town05')

In [80]:
car_tesla = world.get_blueprint_library().filter('*model3*')

# vehicle_all = world.get_blueprint_library().filter('*vehicle*')
# vehicle_tesla = world.try_spawn_actor(car_tesla[0], parking_points[1])

# Expresión regular negativa para excluir múltiples patrones
exclude_regex = re.compile(
    r'.*(ambulance|fusorosa|vespa|kawasaki|crossbike|harley|micro|carlamotors|yamaha|gazelle|diamondback).*')

# Filtrar los vehículos excluyendo los patrones especificados
vehicle_all = [bp for bp in world.get_blueprint_library().filter('*vehicle*') if not exclude_regex.match(bp.id)]


In [81]:
def is_available_parking():
    return np.random.choice([False] + [True] * 2)  #probabily of parking

In [82]:
def generate_parking_points(lines_of_parking_x, lines_of_parking_y):
    parking_points = []
    for x in lines_of_parking_x:
        for y in lines_of_parking_y:
            if is_available_parking():
                parking_points.append(
                    carla.Transform(
                        carla.Location(x=x, y=y, z=1),
                        carla.Rotation(yaw=int(np.random.choice([0, 180])))
                    )
                )
    return parking_points

In [83]:
parking_points = []
lines_of_parking_x = [10, 3.5, -7.0, -13.0, -23, -29]
lines_of_parking_y = [-18.8, -21.6, -24.4, -27.2, -30.0, -32.8, -35.6, -38.4, -41.2, -44.0]

spectator_line = lines_of_parking_x[1]
del lines_of_parking_x[1]
parking_points = generate_parking_points(lines_of_parking_x, lines_of_parking_y)

In [84]:
for point in parking_points:
    ramdom_vehicle = np.random.choice(vehicle_all)
    vehicle = world.try_spawn_actor(ramdom_vehicle, point)
    # print(ramdom_vehicle.id)

In [85]:
# for y in lines_of_parking_y:
check_parking_point = carla.Transform(
    carla.Location(x=spectator_line, y=-30.0, z=1),
    carla.Rotation(yaw=int(np.random.choice([-170, 190])))
)
vehicle_tesla = world.try_spawn_actor(car_tesla[0], check_parking_point)

# vehicle_tesla.destroy()

In [2]:
vehicle_tesla.destroy()

NameError: name 'vehicle_tesla' is not defined

In [86]:

# Especifica las dimensiones de la imagen
ancho_imagen = 1920
alto_imagen = 1080

# Rutas de guardado de las imágenes
hood_image_path = './parking_images/camera_front_hood_image.jpg'
mirror_image_path = './parking_images/camera_front_mirror_image.jpg'

# Obtener la cámara frontal del vehículo
camera_blueprint = world.get_blueprint_library().find('sensor.camera.rgb')
camera_blueprint.set_attribute('image_size_x', str(ancho_imagen))
camera_blueprint.set_attribute('image_size_y', str(alto_imagen))
camera_blueprint.set_attribute('sensor_tick', '0.1')  # Frecuencia de actualización de la cámara en segundos

# Configurar y spawnear la cámara frontal (hood)
camera_location_hood = carla.Location(x=2.5, y=0.0, z=0.5)  # Ajustar la ubicación de la cámara según sea necesario
camera_transform_hood = carla.Transform(camera_location_hood)
camera_hood = world.spawn_actor(camera_blueprint, camera_transform_hood, attach_to=vehicle_tesla)

# Configurar y spawnear la cámara frontal del espejo (mirror)
camera_location_mirror = carla.Location(x=1, y=0.0, z=1.25)  # Ajustar la ubicación de la cámara según sea necesario
camera_transform_mirror = carla.Transform(camera_location_mirror)
camera_mirror = world.spawn_actor(camera_blueprint, camera_transform_mirror, attach_to=vehicle_tesla)

# Definir una función de devolución de llamada para capturar la imagen y cambiar la ubicación de la cámara
def capture_image(image, image_path):
    image_array = np.frombuffer(image.raw_data, dtype=np.uint8)
    image_array = np.reshape(image_array, (image.height, image.width, 4))
    image_bgr = image_array[:, :, :3]  # Convertir de formato RGBA a formato BGR para OpenCV
    
    # Guardar la imagen en la ruta especificada
    cv2.imwrite(image_path, image_bgr)
    cv2.waitKey(1000)
    camera.destroy()

# Conectar la función de devolución de llamada a la cámara frontal (hood)
camera_hood.listen(lambda image: capture_image(image, hood_image_path))

# Conectar la función de devolución de llamada a la cámara frontal del espejo (mirror)
camera_mirror.listen(lambda image: capture_image(image, mirror_image_path))


In [58]:
camera.destroy()
camera_blueprint.destroy()

RuntimeError: trying to operate on a destroyed actor; an actor's function was called, but the actor is already destroyed.

In [4]:
CAMERA_POS_Z = 3
CAMERA_POS_X = -5

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640')  # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '360')
camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z, x=CAMERA_POS_X))

NameError: name 'world' is not defined

In [ ]:
CAMERA_POS_Z = 3
CAMERA_POS_X = -5

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640')
camera_bp.set_attribute('image_size_y', '360')
camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z, x=CAMERA_POS_X))
#this creates the camera in the sim
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle_tesla)


def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))


image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h, image_w, 4))}
# this actually opens a live stream from the camera
camera.listen(lambda image: camera_callback(image, camera_data))

In [5]:
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['image'])

NameError: name 'cv2' is not defined

In [77]:
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()


RuntimeError: trying to operate on a destroyed actor; an actor's function was called, but the actor is already destroyed.

RuntimeError: trying to operate on a destroyed actor; an actor's function was called, but the actor is already destroyed.

RuntimeError: trying to operate on a destroyed actor; an actor's function was called, but the actor is already destroyed.

RuntimeError: trying to operate on a destroyed actor; an actor's function was called, but the actor is already destroyed.

RuntimeError: trying to operate on a destroyed actor; an actor's function was called, but the actor is already destroyed.

...

RuntimeError: trying to operate on a destroyed actor; an actor's function was called, but the actor is already destroyed.

RuntimeError: trying to operate on a destroyed actor; an actor's function was called, but the actor is already destroyed.

RuntimeError: trying to operate on a destroyed actor; an actor's function was called, but the actor is already destroyed.

RuntimeError: trying to operate on a destroyed actor; an actor's function was called, but the actor is already destroyed.

RuntimeError: trying to operate on a destroyed actor; an actor's function was called, but the actor is already destroyed.

In [107]:
vehicle_pos = vehicle_tesla.get_transform()
print(vehicle_pos)

NameError: name 'vehicle_tesla' is not defined

In [19]:
vehicle.set_autopilot(True)